## Checking citations in DH Conference papers

This notebook presents some basic results obtained extracting citations from the abstracts of *(ADHO) DH conferences* from 2015 to 2020 and  from *DHQ journal* articles. The complete dataset is available [here](https://github.com/lehkost/ToolXtractor/)


- [Import Dataset](#Import-the-Dataset)
- [Find DOIs in references](#Find-citations-having-a-DOI)
- [Check DOIs](#Check-DOIs)

    

### Preamble

In [1]:
import ast
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bs4 import BeautifulSoup

In [2]:
def read_tei(tei_file):
    with open(tei_file, 'r') as tei:
        soup = BeautifulSoup(tei, 'lxml')
        return soup
    raise RuntimeError('Cannot generate a soup from the input')

In [3]:
def elem_to_text(elem, default=''):
    if elem:
        return elem.getText(separator=' ', strip=True)
    else:
        return default

In [4]:
from dataclasses import dataclass

@dataclass
class Person:
    firstname: str
    middlename: str
    surname: str

In [138]:
class TEIFile(object):
    def __init__(self, filename):
        self.filename = filename
        self.soup = read_tei(filename)
        self._text = None
        self._title = ''
        self._abstract = ''

    @property
    def doi(self):
        idno_elem = self.soup.find('idno', type='DOI')
        if not idno_elem:
            return ''
        else:
            return idno_elem.getText()

    @property
    def title(self):
        if not self._title:
            if  not self.soup.title:
                self._title = "na"
            else:
                self._title = self.soup.title.getText().replace('\n','').strip()
        return self._title

    @property
    def abstract(self):
        if not self._abstract:
            abstract = self.soup.abstract.getText(separator=' ', strip=True)
            self._abstract = abstract
        return self._abstract

    @property
    def authors(self):
        #authors_in_header = self.soup.analytic.find_all('author')
        authors_in_header = self.soup.find_all('author')

        result = []
        for author in authors_in_header:
            persname = author.persname
            if not persname:
                continue
            firstname = elem_to_text(persname.find("forename"))#, type="first"))
            middlename = elem_to_text(persname.find("forename", type="middle"))
            surname = elem_to_text(persname.surname)
            person = Person(firstname, middlename, surname)
            result.append(person)
        return result
    
    @property
    def bibliography(self):
        bibliography = self.soup.find_all('bibl')
        result = []
        for bibl in bibliography:
            if not bibl:
                continue
            if (elem_to_text(bibl).startswith("DHQ classification")):
                continue
            if (elem_to_text(bibl).startswith("Keywords supplied by author")):
                continue
            if (elem_to_text(bibl).startswith("(accessed )")):
                continue
            if (elem_to_text(bibl).startswith("Enter your references here")):
                continue
            my_bibl_tmp=elem_to_text(bibl).replace('\n','').strip()
            ref = bibl.ref
            lref="";
            ptref="";
            if ref:
                lref=ref['target'].strip();
                #print ('ref '+ref['target'])
            ptr = bibl.ptr
            if ptr:
                ptref=ptr['target'].strip();
                if lref!="" and ("".join(lref.split())).lower()!=("".join(ptref.split()).lower()):
                    lref=lref+" "+ptref;
                    #print ('ptr '+ptr['target'])
            if lref!="" and lref.lower() not in my_bibl_tmp.lower():
                my_bibl_tmp=my_bibl_tmp+" "+("".join(lref.split()))
            
            my_bibl_tmp=my_bibl_tmp.replace(' .', '.')
            #print (my_bibl_tmp)
            result.append(" ".join(my_bibl_tmp.split()))
        return result


    @property
    def text(self):
        if not self._text:
            divs_text = []
            for div in self.soup.body.find_all("div"):
                # div is neither an appendix nor references, just plain text.
                if not div.get("type"):
                    div_text = div.get_text(separator=' ', strip=True)
                    divs_text.append(div_text)

            plain_text = " ".join(divs_text)
            self._text = plain_text
        return self._text

In [139]:
def get_citation_cn (df_dois_values, default=''):
    df_cn_citations = pd.DataFrame (columns = ['doi','cn_citation']);
    import requests;
    headers_dict = {"Accept": "text/x-bibliography", "locale":"en-EN"};
    for vard in df_orig_dois_values:
        if ( vard != "" and vard!=None):
            print(vard)
            try:
                rd =requests.get("http://doi.org/"+vard, headers=headers_dict, timeout=20)
                # print("result: "+r.content.decode("utf-8"))
                if ('DOI Not Found'in rd.text):
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': 'Not Found'}, ignore_index=True)
                else:
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': rd.content.decode("latin-1")}, ignore_index=True)
            except requests.exceptions.ConnectionError:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(503)}, ignore_index=True)
            except requests.exceptions.ConnectTimeout:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
            except requests.exceptions.ReadTimeout:
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
        else:
           # print(var ,0)
            df_cn_citations = df_cn_citations.append({'doi': 'none', 'cn_citation': int(400)}, ignore_index=True)
    return df_cn_citations

In [140]:
def get_citation_from_SSHOC (df_dois_values, default=''):
    df_cn_citations = pd.DataFrame (columns = ['doi','cn_citation']);
    import requests;
    #headers_dict = {"Accept": "text/x-bibliography", "locale":"en-EN"};
    for vard in df_dois_values:
        if ( vard != "" and vard!=None):
            print(vard)
            try:
                rd =requests.get("http://v4e-lab.isti.cnr.it/citationservice/citharvester/getmetadatahtml?pid=https://doi.org/"+vard+"&token=test", timeout=75)
                #print("result: "+rd.content.decode("latin-1"))
                if ('DOI Not Found'in rd.text):
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': 'Not Found'}, ignore_index=True)
                else:
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': rd.content.decode("latin-1")}, ignore_index=True)
            except requests.exceptions.ConnectionError:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(503)}, ignore_index=True)
            except requests.exceptions.ConnectTimeout:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
            except requests.exceptions.ReadTimeout:
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
        else:
           # print(var ,0)
            df_cn_citations = df_cn_citations.append({'doi': 'none', 'cn_citation': int(400)}, ignore_index=True)
    return df_cn_citations

In [141]:
def get_HTTP_Status(urls):
    import requests;
    df_http_status = pd.DataFrame (columns = ['DOI','status'])
    for var in urls:
        #print(var)
        if ( var != ""):
            try:
                r =requests.get("https://doi.org/"+var,timeout=21)
                print("result: "+var+ " ",r.status_code)
                df_http_status = df_http_status.append({'DOI': var, 'status': int(r.status_code)}, ignore_index=True)
            except requests.exceptions.ConnectionError:
              #  print(var)
                df_http_status = df_http_status.append({'DOI': var, 'status': int(503)}, ignore_index=True)
            except requests.exceptions.ConnectTimeout:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(408)}, ignore_index=True)
            except requests.exceptions.ReadTimeout:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(408)}, ignore_index=True)
            except requests.exceptions.RequestException:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(500)}, ignore_index=True)
            except TypeError:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(400)}, ignore_index=True)
        else:
            df_http_status = df_http_status.append({'DOI': var, 'status': int(400)}, ignore_index=True)
    return df_http_status;

In [142]:
import multiprocessing

In [143]:
from os.path import basename, splitext

def basename_without_ext(path):
    base_name = basename(path)
    stem, ext = splitext(base_name)
    if stem.endswith('.tei'):
        # Return base name without tei file
        return stem[0:-4]
    else:
        return stem

In [144]:
def tei_to_csv_entry(tei_file):
    tei = TEIFile(tei_file)
    #print(f"Handled {tei_file}")
    base_name = basename_without_ext(tei_file)
    return base_name, tei.authors, tei.title, tei.bibliography#, tei.abstract

In [145]:
from multiprocessing.pool import Pool
pool = Pool()

### Import the Dataset

The dataset of DH conference abstracts (2016-20020) and DHQ articles is downloaded from https://github.com/lehkost/ToolXtractor/

In [146]:
import glob
from pathlib import Path
papers15 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2015/").glob('*.xml'))
papers16 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2016/").glob('*.xml'))
papers17 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2017/").glob('*.xml'))
papers18 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2018/").glob('*.xml'))
papers19 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2019/").glob('*.xml'))
papers20 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2020/").glob('*.xml'))
papersdhq = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DHQ/").glob('*.xml'))

In [147]:
csv_entries15 = pool.map(tei_to_csv_entry, papers15)
csv_entries16 = pool.map(tei_to_csv_entry, papers16)
csv_entries17 = pool.map(tei_to_csv_entry, papers17)
csv_entries18 = pool.map(tei_to_csv_entry, papers18)
csv_entries19 = pool.map(tei_to_csv_entry, papers19)
csv_entries20 = pool.map(tei_to_csv_entry, papers20)
csv_entriesdhq=pool.map(tei_to_csv_entry, papersdhq)

In [148]:
#test=tei_to_csv_entry("/Users/cesare/git/SSHOCCitationService/data/DHQ/000019.xml")

In [149]:
result_csv0 = pd.DataFrame(csv_entries15, columns=['ID', 'Authors', 'Title', 'Bibliography'])
result_csv1 = result_csv0.append(pd.DataFrame(csv_entries16, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv2 = result_csv1.append(pd.DataFrame(csv_entries17, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv3 = result_csv2.append(pd.DataFrame(csv_entries18, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv4 = result_csv3.append(pd.DataFrame(csv_entries19, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv5 = result_csv4.append(pd.DataFrame(csv_entries20, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv = result_csv5.append(pd.DataFrame(csv_entriesdhq, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv.count()

ID              2788
Authors         2788
Title           2788
Bibliography    2788
dtype: int64

In [150]:
# test_df=result_csv
# test_df['n_references'] = [len(x) for x in test_df['Bibliography']]
# test_df.sort_values('n_references').tail(10)

### Select the papers having the TEI \<bibl\>   elements.

The dataset contains 2788 documents structured according **TEI** document model.

These documents are parsed to individuate those having the *\<bibl\>* (bibliographic citation) element, this element  contains a loosely-structured bibliographic description where the sub-components may or may not be explicitly tagged. There are 1624 documents having this element.

In [151]:
test_df=result_csv
test_df['n_references'] = [len(x) for x in test_df['Bibliography']]
#test_df.sort_values('n_references').tail(10)

In [152]:
test_csv=result_csv[result_csv['Bibliography'].str.len()>1]
#test_csv.count()
test_csv.sort_values('n_references').tail()

,ID,Authors,Title,Bibliography,n_references
171,000172,[],"Digital Humanities, Postfoundationalism, Posti...","[Adams, Melissa et al. Fembot Collective , 200...",113
143,000144,[],Taken Possession of: The Reprinting and Reauth...,[The Celestial Railroad. United States Magazin...,120
76,000077,[],The Digital Future is Now: A Call to Action fo...,"[Ainsworth, P. (2009). Virtual Vellum. Retriev...",130
200,000201,[],The Shock of the Familiar: Three Timelines abo...,"[About AACR2. http://www.aacr2.org/about.html,...",140
110,000111,[],Pertinent Discussions Toward Modeling the Soci...,"[Ovsiannikov, Ilia A., Michael A. Arbib, and T...",254


In [153]:
#com_test_csv[com_test_csv['n_references']==140].head().style.set_properties(subset=['Bibliography'], **{'width': '600px'})

In [154]:
# all citations
my_df=test_csv[['ID','Title','Bibliography']]
my_exp_df_tmp_1=my_df.explode('Bibliography')
# cleaning...
my_exp_df= my_exp_df_tmp_1[(~my_exp_df_tmp_1['Bibliography'].str.startswith("DHQ classification")) & (~my_exp_df_tmp_1['Bibliography'].str.startswith("Keywords supplied by author"))
                                & (~my_exp_df_tmp_1['Bibliography'].str.startswith("(accessed")) & (~my_exp_df_tmp_1['Bibliography'].str.startswith("Enter your references here"))]

my_exp_df.count()

ID              25115
Title           25115
Bibliography    25115
dtype: int64

In [155]:
my_exp_tmp_df=my_exp_df[my_exp_df['Bibliography'].str.len()>0]
my_exp_tmp_df.sort_values("Bibliography").head()

,ID,Title,Bibliography
179,000180,Escaping the Shallows: Deep Reading’s Revival ...,#1book140 (2014). #Outlander. 10 February. Ava...
197,000198,Humanities Unbound: Supporting Careers and Sch...,#Alt-Academy (January 2011): http://mediacommo...
322,WALSH_Brandon_Digital_Humanities_Pedagogy_and_...,Digital Humanities Pedagogy and Praxis Roundtable,"#FutureEd. Humanities, Arts, Science, and Tech..."
207,000208,Beyond the Margins: Intersectionality and the ...,#transformDH. About #transformDH. http://origi...
242,PUREN_M_P__The_Time_Us_project__Creating_gold_...,The Time-Us project. Creating gold data to und...,(2009). Extracting and Visualizing Quotations ...


In [156]:
my_exp_tmp_df.count()

ID              25096
Title           25096
Bibliography    25096
dtype: int64

In [157]:
my_exp_df_tmp=my_exp_df[my_exp_df['Bibliography'].str.len()>0]
my_exp_df_tmp['Bibliography']=my_exp_df_tmp['Bibliography']
df_temp_dupl=my_exp_df_tmp[my_exp_df_tmp.duplicated('Bibliography', keep='last')]
#df_temp_dupl.sort_values('Bibliography').iloc[0:30]
df_temp_dupl.count()

/Users/cesare/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ID              325
Title           325
Bibliography    325
dtype: int64

The total number of citations individuated in the dataset is 25096.

### Find citations having a DOI
*Regular expressions* are used to parse all citations in order to individuate those having a DOI.

In [158]:
import re

regexu1=re.compile(
       r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"
       )
df_refs=my_exp_df.Bibliography.values
df_refs_with_url = pd.DataFrame(columns=["Reference", "URL"])
ureferences=[]
URLs=[]
for ureference in df_refs:
    myur=re.search(regexu1, ureference)
    if myur:
        ureferences.append(ureference);
        URLs.append(myur[0]);
        #print (ureference+'\n'+myur[0], end="\n\n")
    #else:
        #print (reference)
df_refs_with_url['Reference']=ureferences;
df_refs_with_url['URL']=URLs;
df_refs_with_url.drop_duplicates(inplace=True)
#df_refs_with_url.info()
refwURL=df_refs_with_url.count()
print (f'\n There are {refwURL[1]} references with a URL \n')


 There are 6838 references with a URL 



In [159]:
import re
regex = re.compile(r'\b(10[.][0-9]{4,}(?:[.][0-9]+)*/(?:(?!["&\'<>])\S)+)\b', re.IGNORECASE)
#regex = re.compile(r'\b(10.\d{0,}/[-._;()/:A-Z0-9])\b', re.IGNORECASE)
#regex = re.compile(
#        r'/^10.\d{4,9}/[-._;()/:A-Z0-9]+$/i'
#        r'/^10.1002/[^\s]+$/i'
#        r'/^10.\d{4}/\d+-\d+X?(\d+)\d+<[\d\w]+:[\d\w]*>\d+.\d+.\w+;\d$/i'
#        r'/^10.1021/\w\w\d+$/i'
#        r'/^10.1207/[\w\d]+\&\d+_\d+$/i', re.IGNORECASE)
df_refs=my_exp_df.Bibliography.values
df_refs_with_doi = pd.DataFrame(columns=["Reference", "DOI"])
references=[]
DOIs=[]
for reference in df_refs:
    mydoi=re.search(regex, reference)
    if mydoi:
        references.append(reference);
        DOIs.append(mydoi[0]);
        #print (reference+'\n'+mydoi[0], end="\n\n")
df_refs_with_doi['Reference']=references;
df_refs_with_doi['DOI']=DOIs;
df_refs_with_doi.drop_duplicates(inplace=True)
refwDOI=df_refs_with_doi.count()
print (f'\n There are {refwDOI[1]} references with a DOI \n')


 There are 1173 references with a DOI 



In [160]:
#Example: five citations that have DOIs
df_refs_with_doi.head()
df_refs_with_doi.head().style.set_properties(subset=['Reference'], **{'width': '600px'})

,Reference,DOI
0,"Byrne, G., and Goddard, L. (2010). The Strongest Link: Libraries and Linked Data. D-Lib Magazine , 16 (11/12), doi:10.1045/november2010-byrne.",10.1045/november2010-byrne
1,"Lampert, C. K., and Southwick, S. B. (2013). Leading to Linking: Introducing Linked Data to Academic Library Digital Collections. Journal of Library Metadata , 13 (2–3): 230–53, doi:10.1080/19386389.2013.826095.",10.1080/19386389.2013.826095
2,"Singer, R. (2009). Linked Library Data Now! Journal of Electronic Resources Librarianship , 21 (2): 114–26, doi:10.1080/19411260903035809.",10.1080/19411260903035809
3,"Thomas, L. and Solomon, D. (2014). Active Users: Project Development and Digital Humanities Pedagogy. CEA Critic, 76 (2) (July): 211–20, DOI:10.1353/cea.2014.0014.",10.1353/cea.2014.0014
4,"Farquhar, A. and Baker, J. (2014). Interoperable Infrastructures for Digital Research: A Proposed Pathway for Enabling Transformation. Digital Humanities 2014, http://dx.doi.org/10.6084/m9.figshare.1092550%20.",10.6084/m9.figshare.1092550%20


### Check DOIs

To check if there are errors, the DOIs are invoked and the HTTP Return Status is checked.

In [ ]:
df_http_status=get_HTTP_Status(df_refs_with_doi.DOI.values)

#### DOIs HTTP Return Status: 404 Not Found

References where DOIs return *404 Not Found*

In [162]:
df_404=df_http_status[df_http_status['status'] == 404]
test=df_404.join(df_refs_with_doi.set_index('DOI'), on='DOI')
test1=test.drop_duplicates(subset='DOI', keep="last")
test1.count()

DOI          30
status       30
Reference    30
dtype: int64

In [164]:
#example of DOIs returning 404
test.drop_duplicates(subset='DOI', keep='first').reset_index(drop=True).tail(4).style.set_properties(subset=['Reference'], **{'width': '600px'})
#test.to_csv(path_or_buf='../data/notfound.csv', sep=';')

,DOI,status,Reference
26,10.1002/9781405177504.ch28/summary,404,"Hoover, D. L. (2013). Quantitative Analysis and Literary Studies. In R. Siemens & S. Schreibman (Eds.), A Companion to Digital Literary Studies (pp. 517–533). John Wiley & Sons, Ltd. Retrieved from http://onlinelibrary.wiley.com/doi/10.1002/9781405177504.ch28/summary"
27,10.1002/9780470999875.ch,404,"McCarty, Willard. Modeling. A Study in Words and Meanings. In Schreibman, Susan, Siemens, Ray et Unsworth, John (éd.). A Companion to Digital Humanities. Willey (2004). DOI: 10.1002/9780470999875.ch 19"
28,10.1207/s15327051hci1304,404,"Dourish, Paul, and Graham Button. 1998. On ‘Technomethodology’: Foundational Relationships between Ethnomethodology and System Design. Human-Computer Interaction 13 (4): 395–432. https://doi.org/10.1207/s15327051hci1304."
29,10.1007/978-3-642-39209-2-20,404,"Al-Omar, M., & Cox, A. (2013). Finders, Keepers, Losers, Seekers: A Study of Academics' Research-Related Personal Information Collections. Lecture Notes in Computer Science, 8016 LNCS (PART 1), 169–176. doi:10.1007/978-3-642-39209-2-20"


#### Retrieve citation DOIs using CrossRef API
Crossref API allows [querying the database by giving it in input strings that contain bibliography references](https://www.crossref.org/labs/resolving-citations-we-dont-need-no-stinkin-parser/). The reference string does not to be necessarily a well-written references. The input string is parsed by Crossref using machine learning techniques and the system tries to match the reference string with the metadata that are stored in the database. 

An important feature of Crossref API, is the score of sureness that Crossref API retrieve beside the document’s metadata. For each request, Crossref score indicates how much it is sure about the entities retrieved, if the score value is high the metadata retrieved are probably correct, if the score is low the metadata retrieved might be the wrong ones.

Here the Crossref query API is used to check if citations whose DOI http connection returns *404 Not Found* can be found in the Crossref Database and, if yes, to find what is the correct DOI. The value *110* is the minimum score value.

In [ ]:
import urllib.request, json
references=test1['Reference'];
df_citations404 = pd.DataFrame(columns=["Publication_cit", "Crossref_cit_title", "Crossref_cit_author", "Crossref_cit_metadata", "Crossref_DOI"])
originalCitations=[]
crossrefCitations=[]
crossrefCitationsAuthor=[]
crossrefCitationsMetadata=[]
DOIs=[]
score=[]
i=0;
j =0;
for citenf in references:
    cit=urllib.parse.quote_plus(citenf)
    try:
        with urllib.request.urlopen("https://api.crossref.org/works?query.bibliographic="+cit+"&sort=score&mailto=cesare.concordia@gmail.com#", timeout=18000) as url:
            data16 = json.loads(url.read().decode())
            j=j+1
            if (j%5 == 0):
                print(f"{j}, ({i})")
            if (len(data16["message"]["items"])>0) and (data16["message"]["items"][0]['score'] >110):
                originalCitations.append(citenf)
                crossrefCitationsMetadata.append( data16["message"]["items"][0])
                crossrefCitations.append( data16["message"]["items"][0]['title'])
                crossrefCitationsAuthor.append( data16["message"]["items"][0]['author'])
                DOIs.append(data16["message"]["items"][0]['DOI'])
                score.append(data16["message"]["items"][0]['score'])
                i=i+1
                #print(f"{i} found, out of {j}")
            if (j>1000):
                break
    except urllib.error.URLError:
        print(cit)
    except urllib.error.HTTPError:
        print(cit)
        
df_citations404["Publication_cit"] = originalCitations
df_citations404["Crossref_cit_title"] = crossrefCitations
df_citations404["Crossref_cit_author"] = crossrefCitationsAuthor
df_citations404["Crossref_cit_metadata"] = crossrefCitationsMetadata
df_citations404["Crossref_DOI"] = DOIs
df_citations404["Score"] = score
#df_citations404.head()

Using the Crossref API, 10 DOIs have been found

In [166]:
df_citations404.count()
#df_citations404[['Publication_cit', 'Crossref_cit_title', 'Crossref_cit_author', 'Crossref_DOI']].head(10).style.set_properties(subset=['Publication_cit'], **{'width': '200px'})

Publication_cit          10
Crossref_cit_title       10
Crossref_cit_author      10
Crossref_cit_metadata    10
Crossref_DOI             10
Score                    10
dtype: int64

In [169]:
#df_citations404.head()

In [170]:
retrieved404=df_citations404.join(test.set_index('Reference'), on='Publication_cit',  lsuffix='_in_publication')
retrieved404[['Publication_cit', 'Crossref_cit_title', 'Crossref_cit_author', 'DOI', 'Crossref_DOI']].head(10).style.set_properties(subset=['Publication_cit','Crossref_cit_title'], **{'width': '200px'})

,Publication_cit,Crossref_cit_title,Crossref_cit_author,DOI,Crossref_DOI
0,"Griffiths Mary and Kim Barbour. ""‘Imagine If Our Cities Talked to Us’: Questions about the Making of ‘responsive’ Places and Urban Publics."" In Making Publics, Making Places, 27-48. South Australia: University of Adelaide Press, 2016, http://www.jstor.org/stable/10.20851/j.ctt1t304qd.8","[""'Imagine if our cities talked to us': Questions about the making of 'responsive' places and urban publics""]","[{'given': 'Mary', 'family': 'Griffiths', 'sequence': 'first', 'affiliation': []}]",10.20851/j.ctt1t304qd.8,10.20851/publics-03
1,"Underwood, T., Bamman, D. and Lee, S. (2018). The transformation of Gender in English language fiction. Cultural Analytics, http://culturalanalytics.org/2018/02/thetransformation-of-gender-in-english-language-fiction/ (DOI: 10.7910/DVN/TEGMGI)",['The Transformation of Gender in English-Language Fiction'],"[{'given': 'Ted', 'family': 'Underwood', 'sequence': 'first', 'affiliation': []}, {'given': 'David', 'family': 'Bamman', 'sequence': 'additional', 'affiliation': []}, {'given': 'Sabrina', 'family': 'Lee', 'sequence': 'additional', 'affiliation': []}]",10.7910/DVN/TEGMGI,10.22148/16.019
2,"Bates, M. (1998). Indexing and access for digital libraries and the Internet: human, database, and domain factors. Journal of the American Society for Information Science , 49 (13) doi:10.1002/(SICI)1097-4571(1998110)49:13<1185::AID-ASI6>3.3.CO;2-M (accessed 23 March 2011).","['Indexing and access for digital libraries and the internet: Human, database, and domain factors']","[{'given': 'Marcia J.', 'family': 'Bates', 'sequence': 'first', 'affiliation': []}]",10.1002/(SICI)1097-4571(1998110)49:13,10.1002/(sici)1097-4571(1998110)49:13<1185::aid-asi6>3.0.co;2-v
3,"Spiro, L. (2012). “This Is Why We Fight”: Defining the Values of the Digital Humanities. In M. K. Gold (ed.), Debates in the Digital Humanities. University of Minnesota Press, pp. 16–35. Available at http://minnesota.universitypressscholarship.com/view/10.5749/minnesota/9780816677948.001.0001/upso-9780816677948-chapter-3",['“This Is Why We Fight”: Defining the Values of the Digital Humanities'],"[{'given': 'Lisa', 'family': 'Spiro', 'sequence': 'first', 'affiliation': []}]",10.5749/minnesota/9780816677948.001.0001/upso-9780816677948-chapter-3,10.5749/minnesota/9780816677948.003.0003
4,"Shapiro, L. A. (2012). Embodied Cognition. In Margolis, E., Samuels, R., and Stich, S. P. (eds.), The Oxford Handbook of Philosophy of Cognitive Science. Oxford, New York: Oxford University Press, pp. 118–46. http://www.oxfordhandbooks.com/view/10.1093/oxfordhb/9780195309799.001.0001/oxfordhb-9780195309799-e-6 (accessed 26 November 2018).",['6. Embodied Cognition'],"[{'given': 'Lawrence A.', 'family': 'Shapiro', 'sequence': 'first', 'affiliation': []}]",10.1093/oxfordhb/9780195309799.001.0001/oxfordhb-9780195309799-e-6,10.1093/oxfordhb/9780195309799.003.0006
5,"Hirsch, Brett D., 2011. The Kingdom Has Been Digitized: Electronic Editions of Renaissance Drama and the Long Shadows of Shakespeare and Print. Literature Compass [online] 8 (9): 568-591. Available from: http://onlinelibrary.wiley.com/doi/10.1111/lico.2011.8.issue-9/issuetoc [Accessed 27 September 2012].",['The Kingdom has been Digitized: Electronic Editions of Renaissance Drama and the Long Shadows of Shakespeare and Print'],"[{'given': 'Brett D.', 'family': 'Hirsch', 'sequence': 'first', 'affiliation': []}]",10.1111/lico.2011.8.issue-9/issuetoc,10.1111/j.1741-4113.2011.00830.x
6,"Romanello, Matteo. 2013. Creating an Annotated Corpus for Extracting Canonical Citations from Classics-Related Texts by Using Active Annotation. In Computational Linguistics and Intelligent Text Processing. 14th International Conference, CICLing 2013, Samos, Greece, March 24-30, 2013, Proceedings, Part I , edited by Alexander Gelbukh, 1:60–76. Lecture Notes in Computer Science / Theoretical Computer Science and General Issues. Springer Berlin Heidelberg. doi:10.1007/9

In [171]:
retrieved404.Crossref_DOI.values

array(['10.20851/publics-03', '10.22148/16.019',
       '10.1002/(sici)1097-4571(1998110)49:13<1185::aid-asi6>3.0.co;2-v',
       '10.5749/minnesota/9780816677948.003.0003',
       '10.1093/oxfordhb/9780195309799.003.0006',
       '10.1111/j.1741-4113.2011.00830.x', '10.1007/978-3-642-37247-6_6',
       '10.1002/9781118680605.ch23', '10.1002/9781405177504.ch28',
       '10.1207/s15327051hci1304_2'], dtype=object)

In [172]:
dfdc=get_HTTP_Status(retrieved404.Crossref_DOI.values)

result: 10.20851/publics-03  200
result: 10.22148/16.019  200
result: 10.1002/(sici)1097-4571(1998110)49:13<1185::aid-asi6>3.0.co;2-v  403
result: 10.1111/j.1741-4113.2011.00830.x  403
result: 10.1007/978-3-642-37247-6_6  200
result: 10.1002/9781118680605.ch23  403
result: 10.1002/9781405177504.ch28  403
result: 10.1207/s15327051hci1304_2  200


In [173]:
dfdc_st = dfdc['status'].value_counts()
dfdc_st.head()

403    4
200    4
408    2
Name: status, dtype: int64

8 retrieved DOIs are valid (HTTP 403 is a HTTP status code meaning that he server understood the request, but will not fulfill it due to client-related issues), 2 of them returns connection timeout status.

In [ ]:
dfdc.head(10)

In [ ]:
dfdc_t=dfdc.join(retrieved404[['Publication_cit', 'Crossref_DOI']].set_index('Crossref_DOI'), on='DOI',  lsuffix='_')
dfdc_t.head(10).style.set_properties(subset=['Publication_cit'], **{'width': '300px'})